<a href="https://colab.research.google.com/github/Hidenori24/LearnDirectory/blob/master/SMBC2025_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. ライブラリセットアップ


In [36]:
# ============================================
# 0. ライブラリ & CFG 定義
# ============================================
!pip install -q xgboost optuna japanize-matplotlib folium

import os, warnings, random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import folium
from xgboost import XGBRegressor
import optuna
from sklearn.metrics import mean_squared_error, r2_score

warnings.filterwarnings("ignore")

# ---------- CFG ----------
class CFG:
    seed         = 42

    data_path    = '/content/drive/MyDrive/ML/Signate_1634/'
    corr_threshold = 0.13                 # 相関係数の閾値
    comfort_index  = 65                   # 不快指数の快適基準
    optuna_trials  = 50                   # Optuna 試行回数


# set seed
random.seed(CFG.seed)
np.random.seed(CFG.seed)


# 1. Google Drive マウント


In [37]:
# ============================================
# 1. Google Drive マウント
# ============================================
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


# 2. データ読み込み

In [38]:
# =========================================================
# 2. データ読み込み
#    - index を DatetimeIndex（UTC）に
# =========================================================
def read_data(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)                 # まずは普通に読み込み
    df['time'] = pd.to_datetime(df['time'], utc=True)   # ①文字列→datetime(UTC)
    df['time'] = df['time'].dt.tz_convert(None)         # ②タイムゾーン情報を外す（naive へ）
    df = df.set_index('time').sort_index()              # ③DatetimeIndex として設定
    return df

train_df = read_data(os.path.join(CFG.data_path, 'train.csv'))
test_df  = read_data(os.path.join(CFG.data_path, 'test.csv'))

# タイムゾーンを UTC→Etc/GMT-1 に変換
train_df.index = pd.to_datetime(train_df.index, utc=True).tz_convert('Etc/GMT-1')
test_df.index = pd.to_datetime(test_df.index,  utc=True).tz_convert('Etc/GMT-1')

print('train', train_df.shape, 'test', test_df.shape)



train (26280, 91) test (8760, 90)


#3 前処理

In [39]:
import holidays
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import lightgbm as lgb

# weather_main, weather_icon, weather_id 列の削除
for kw in ['weather_main','weather_icon','weather_id']:
    cols = [c for c in train_df.columns if kw in c]
    train_df.drop(columns=cols, inplace=True)
    test_df.drop(columns=cols, inplace=True)

# 値がすべて同じ列の削除
constant_cols = [c for c in train_df.columns if train_df[c].nunique() == 1]
train_df.drop(columns=constant_cols, inplace=True)
test_df.drop(columns=constant_cols, inplace=True)

# 大気圧のクリッピング（1090 hPa で上限）
pressure_cols = [c for c in train_df.columns if 'pressure' in c]
train_df[pressure_cols] = train_df[pressure_cols].clip(upper=1090)
test_df [pressure_cols] = test_df [pressure_cols].clip(upper=1090)

# 風速のクリッピング（18 m/s で上限）
wind_cols = [c for c in train_df.columns if 'wind_speed' in c]
train_df[wind_cols] = train_df[wind_cols].clip(upper=18)
test_df [wind_cols] = test_df [wind_cols].clip(upper=18)

# 欠損値を前方埋め、後方埋め
train_df.ffill(inplace=True); train_df.bfill(inplace=True)
test_df .ffill(inplace=True); test_df .bfill(inplace=True)

# 4. 不快指数 → 差分絶対値

In [40]:
def discomfort(temp, hum):
    return temp * 0.81 + (hum/100)*(temp-14.3) + 46.3

for city in ['valencia','madrid','bilbao','barcelona','seville']:
    # 生不快指数
    train_df[f'di_{city}'] = discomfort(train_df[f'{city}_temp'], train_df[f'{city}_humidity'])
    test_df [f'di_{city}'] = discomfort(test_df [f'{city}_temp'], test_df [f'{city}_humidity'])
    # 快適基準との差分絶対値
    train_df[f'di_{city}_diff_abs'] = (train_df[f'di_{city}'] - CFG.comfort_index).abs()
    test_df [f'di_{city}_diff_abs'] = (test_df [f'di_{city}'] - CFG.comfort_index).abs()

# 元の temp, humidity, 生不快指数 列は不要なので削除
drop_cols = [c for c in train_df.columns if ('_temp' in c or '_humidity' in c or (c.startswith('di_') and not c.endswith('_diff_abs')))]
train_df.drop(columns=drop_cols, inplace=True)
test_df .drop(columns=drop_cols, inplace=True)

# ============================================
# 5. generation_sum を追加
# ============================================
gen_cols = [c for c in train_df.columns if c.startswith('generation')]
train_df['generation_sum'] = train_df[gen_cols].sum(axis=1)
test_df ['generation_sum'] = test_df [gen_cols].sum(axis=1)

# ============================================
# 6. 相関係数で特徴量選択（数値列のみで計算）
# ============================================
# 数値列だけを抽出
numeric_cols = train_df.select_dtypes(include=[np.number]).columns

# 数値列同士の相関行列を計算し、price_actual との相関を取得
corr = train_df[numeric_cols].corr()['price_actual'].abs()

# 閾値以上かつ price_actual 自身は除外
use_cols = corr[corr > CFG.corr_threshold].index.drop('price_actual').tolist()
print("選択された特徴量:", use_cols)

# ============================================
# 7. 日付情報の追加 & One-Hot
# ============================================
for df in [train_df, test_df]:
    df.index = pd.to_datetime(df.index)  # 念のため
    df['month']   = df.index.month       # ←ここを追加！
    df['hour']    = df.index.hour
    df['weekday'] = df.index.dayofweek
    df['day_cat'] = df.index.day.map(lambda d: 0 if d <= 10 else 1 if d <= 20 else 2)
# month は one-hot に
train_df = pd.get_dummies(train_df, columns=['month'], prefix='month', drop_first=True)
test_df  = pd.get_dummies(test_df,  columns=['month'], prefix='month', drop_first=True)

選択された特徴量: ['generation_biomass', 'generation_fossil_brown_coal/lignite', 'generation_fossil_gas', 'generation_fossil_hard_coal', 'generation_fossil_oil', 'generation_hydro_pumped_storage_consumption', 'generation_hydro_run_of_river_and_poundage', 'total_load_actual', 'valencia_wind_speed', 'madrid_wind_speed', 'bilbao_pressure', 'bilbao_wind_speed', 'bilbao_clouds_all', 'barcelona_pressure', 'barcelona_wind_speed', 'seville_pressure', 'seville_wind_deg', 'generation_sum']


# 8.学習／検証データの準備

In [41]:
X = train_df[use_cols + ['hour','weekday','day_cat'] + [c for c in train_df.columns if c.startswith('month_')]]
y = train_df['price_actual']
X_test = test_df[X.columns]  # テストデータの列順を合わせる

X_train = X[train_df.index.year < 2017]
y_train = y[train_df.index.year < 2017]
X_val   = X[train_df.index.year == 2017]
y_val   = y[train_df.index.year == 2017]

print("train:", X_train.shape, "val:", X_val.shape, "test:", X_test.shape)

train: (17520, 32) val: (8760, 32) test: (8760, 32)


# 9. Optuna によるパラメータチューニング

In [43]:
def objective(trial):
    params = {
        'objective':        'reg:quantileerror',
        'eval_metric':      'rmse',
        'learning_rate':    trial.suggest_float('learning_rate', 0.001, 0.2),
        'n_estimators':     trial.suggest_int('n_estimators', 100, 1000),
        'max_depth':        trial.suggest_int('max_depth', 5, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma':            trial.suggest_float('gamma', 0.001, 0.1),
        'subsample':        trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'lambda':           trial.suggest_float('lambda', 0.001, 1.0),
        'quantile_alpha':   trial.suggest_float('quantile_alpha', 0.7, 0.9),
        'verbosity':        0,
        'random_state':     CFG.seed
    }
    model = XGBRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    # RMSE を自前で計算
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    return rmse

study = optuna.create_study(direction='minimize',
                            sampler=optuna.samplers.TPESampler(seed=CFG.seed))
study.optimize(objective, n_trials=CFG.optuna_trials)
best_params = study.best_params
print("Best Params:", best_params)

[I 2025-06-17 04:01:17,143] A new study created in memory with name: no-name-6d3bde41-a703-4bf8-8f17-8cf395f05840
[I 2025-06-17 04:01:28,572] Trial 0 finished with value: 9.769733762645801 and parameters: {'learning_rate': 0.07553348365062514, 'n_estimators': 956, 'max_depth': 9, 'min_child_weight': 6, 'gamma': 0.016445845403801215, 'subsample': 0.5779972601681014, 'colsample_bytree': 0.5290418060840998, 'lambda': 0.8663099696291603, 'quantile_alpha': 0.8202230023486418}. Best is trial 0 with value: 9.769733762645801.
[I 2025-06-17 04:01:32,571] Trial 1 finished with value: 9.461384977581444 and parameters: {'learning_rate': 0.14190644298141306, 'n_estimators': 118, 'max_depth': 10, 'min_child_weight': 9, 'gamma': 0.022021571957149343, 'subsample': 0.5909124836035503, 'colsample_bytree': 0.5917022549267169, 'lambda': 0.3049380007165782, 'quantile_alpha': 0.8049512863264475}. Best is trial 1 with value: 9.461384977581444.
[I 2025-06-17 04:01:36,685] Trial 2 finished with value: 10.61644

Best Params: {'learning_rate': 0.012595967756143741, 'n_estimators': 501, 'max_depth': 5, 'min_child_weight': 5, 'gamma': 0.0855218837128587, 'subsample': 0.9905858339222093, 'colsample_bytree': 0.5615546708760337, 'lambda': 0.6255144534471777, 'quantile_alpha': 0.8116417816317152}


# 10. ベストパラメータで最終モデル学習 → テスト予測


In [44]:
model = XGBRegressor(
    objective='reg:quantileerror',
    eval_metric='rmse',
    verbosity=0,
    random_state=CFG.seed,
    **best_params
)
model.fit(X, y)
y_test_pred = model.predict(X_test)

# 11. Submission ファイル出力

In [45]:
sub = pd.read_csv(CFG.data_path + 'sample_submit.csv', header=None)
sub[1] = y_test_pred
sub.to_csv('submission.csv', header=False, index=False)
print(">>> submission.csv を出力しました")

>>> submission.csv を出力しました
